<a href="https://colab.research.google.com/github/MemphisMeng/BigQuery-AI---Building-the-Future-of-Data/blob/main/Spam%20Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparation

## necessary packages
---
It is necessary to make sure the libraries are compatible with each other, therefore, the versions are important.

In [1]:
# Install all necessary libraries, including BigQuery client
!pip install "bigframes==2.9.0"
!pip install "google-cloud-bigquery[bqstorage,pandas]==3.33.0"
!pip install "google-cloud-bigquery-storage==2.30.0"
!pip install "pandas==2.2.2"
!pip install "protobuf==5.29.1" "rich==13.9.2"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.1/447.1 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 43.8 MB/s eta 0:00:00
  Attempting uninstall: bigframes
    Found existing installation: bigframes 2.18.0
    Uninstalling bigframes-2.18.0:
      Successfully uninstalled bigframes-2.18.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.5/253.5 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-bigquery
    Found existing installation: google-cloud-bigquery 3.36.0
    Uninstalling google-cloud-bigquery-3.36.0:
      Successfully uninstalled google-cloud-bigquery-3.36.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-bigquery-storage
    Found existing installation: google-cloud-bigquery-storage 2.33.0
    Uninstalling google-cloud-bigquery-storage-2.33.0:
      Successfully uninstalled google-cloud-bigquery-storage-2.33.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 19.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: rich
    Found existing installation: rich 13.9.4
    Uninstalling rich-13.9.4:
      Successfully uninstalled rich-13.9.4


## datasets


---
The referenced dataset is publicly accessible on kaggle, credit ML team in UCI.


In [2]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
from kagglehub import KaggleDatasetAdapter
kagglehub.login()

Kaggle credentials set.
Kaggle credentials successfully validated.


In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
# input datasets
import pandas as pd

dataset_path = 'uciml/sms-spam-collection-dataset'
dataset_directory = kagglehub.dataset_download(dataset_path)
sms_df = pd.read_csv(f"{dataset_directory}/spam.csv", encoding="latin1")

100%|██████████| 211k/211k [00:00<00:00, 45.1MB/s]

Extracting files...


In [5]:
# data cleansing
## combine all content from the 2nd to 5th column
sms_df['message'] = None
sms_df['message'].loc[sms_df['Unnamed: 4'].notna()] = \
  sms_df['v2'] + ',' + sms_df['Unnamed: 2'].astype(str) + ',' + sms_df['Unnamed: 3'].astype(str) + ',' + sms_df['Unnamed: 4'].astype(str)

sms_df['message'].loc[(sms_df['Unnamed: 4'].isna()) & (sms_df['Unnamed: 3'].notna())] = \
  sms_df['v2'] + ',' + sms_df['Unnamed: 2'].astype(str) + ',' + sms_df['Unnamed: 3'].astype(str)

sms_df['message'].loc[(sms_df['Unnamed: 3'].isna()) & (sms_df['Unnamed: 2'].notna())] = \
  sms_df['v2'] + ',' + sms_df['Unnamed: 2'].astype(str)

sms_df['message'].loc[sms_df['Unnamed: 2'].isna()] = sms_df['v2']

# rename the column to make it meaningful
sms_df.rename(columns={'v1': 'label'}, inplace=True)

# take the necessary columns
sms_df = sms_df[['message', 'label']]

In [6]:
# # Link to BigQuery
# # "Add-ons -> Google Cloud Services - BigQuery"

# from google.cloud import bigquery
PROJECT_ID = "chatbot-278522"  # ⚠️ CHANGE THIS to your actual Google Cloud project ID
# DATASET_ID = "retail_Business_Intelligence"        # This will be created for you
# client = bigquery.Client(project=PROJECT_ID)

# Model Training

In [7]:
from bigframes.ml.llm import GeminiTextGenerator, TextEmbeddingGenerator
import bigframes
bigframes.options.bigquery.project = PROJECT_ID

## X


---

The plain text is going to be vectorized.

In [8]:
embedder = TextEmbeddingGenerator()
embedded_X = embedder.predict(sms_df['message'])

## y
---
As this is a classification challenge, the output column is going to be label encoded.

In [9]:
from bigframes.ml.preprocessing import LabelEncoder

transformer = LabelEncoder(max_categories=2)
transformed_y = transformer.fit_transform(sms_df['label'])

## Discover the most suitable hyperparameter configuration

In [ ]:
from bigframes.ml.ensemble import XGBClassifier
from bigframes.ml.model_selection import cross_validate

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2]
}

best_score = -float('inf')
best_params = None

# Iterate through the parameter grid
for n_estimators in param_grid['n_estimators']:
    for max_depth in param_grid['max_depth']:
        for learning_rate in param_grid['learning_rate']:
            print(f"Evaluating params: n_estimators={n_estimators}, max_depth={max_depth}, learning_rate={learning_rate}")

            # Initialize the XGBoost classifier with current parameters
            clf = XGBClassifier(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate)

            # Perform cross-validation
            cv_results = cross_validate(
                estimator=clf,
                X=embedded_X['ml_generate_embedding_result'],
                y=transformed_y['labelencoded_label'],
                cv=5  # Using 5-fold cross-validation
            )

            # Calculate the average cross-validation score
            mean_score = pd.Series(cv_results['test_score']).mean()

            print(f"  Average CV score: {mean_score}")

            # Check if the current parameters give a better score
            if mean_score > best_score:
                best_score = mean_score
                best_params = {
                    'n_estimators': n_estimators,
                    'max_depth': max_depth,
                    'learning_rate': learning_rate
                }

# Print the best parameters and best score
print("\nBest parameters found: ", best_params)
print("Best average cross-validation score: ", best_score)

Evaluating params: n_estimators=100, max_depth=3, learning_rate=0.01
